# 2. AI Text generation (Generación de texto artificial)

* **Objetivo:** Crear el "Dataset Sombra" (Shadow Dataset).
* **Estrategia:** Utilizar la API de **Google Gemini** para reescribir cada noticia del dataset humano original. El objetivo es mantener los hechos informativos pero alterar el estilo sintáctico y léxico para que refleje los patrones de una IA.

## 2.1. Configuración inicial

Importamos las librerías necesarias y definimos las constantes globales.
* Definimos `INPUT_FILE` (la muestra humana creada en el Notebook 1) y `OUTPUT_FILE` (donde se guardará el dataset pareado de IA).

In [1]:
#%pip install google-generativeai

In [ ]:
from google import generativeai as genai
import pandas as pd
import time
import os

# API Key de Google Gemini (PERSONAL)
GOOGLE_API_KEY = "API_KEY_AQUI"

# Rutas de archivos
INPUT_FILE = '../data/1_raw/all-the-news-5k-sample.csv'
OUTPUT_FILE = '../data/3_synthetic/ai_generated_gemini.csv'

# Crear carpeta de salida si no existe
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

## 2.2. Inicialización del cliente Gemini

Configuramos el cliente con la clave API y seleccionamos el modelo.
Usamos **`gemini-2.0-flash`**, **`gemini-2.5-flash-lite-preview-09-2025`** por ser el modelo más eficiente (rápido y gratuito) para tareas de reescritura masiva.

In [3]:
# Configurar la API
genai.configure(api_key=GOOGLE_API_KEY)

# Instanciar el modelo
model = genai.GenerativeModel('gemini-2.5-flash-lite-preview-09-2025')

print("✅ Cliente Gemini configurado correctamente.")

✅ Cliente Gemini configurado correctamente.


## 2.3. Definición de la función de reescritura

Definimos la función `rewrite_article`.
* **Prompt Engineering:** Instruimos al modelo para que actúe como un "Periodista IA". Le pedimos explícitamente que mantenga la información pero use su propio estilo.
* **Limpieza:** Solicitamos que no añada introducciones ("Here is the text..."), solo el cuerpo del artículo.

In [4]:
def rewrite_article(text):
    """
    Envía el texto a Gemini para ser reescrito con estilo sintético.
    """
    # Prompt diseñado para imitar estilo periodístico artificial
    prompt = f"""
    Act as an AI journalist. Rewrite the following news article text.
    Maintain the core information and facts, but use your own sentence structure and vocabulary.
    Do not output any introduction like "Here is the rewritten text". Output ONLY the article body.
    
    Original Text:
    {text[:8000]} 
    """
    
    # Generación de contenido
    response = model.generate_content(prompt)
    
    # Retornamos el texto limpio
    return response.text.strip()

## 2.4. Bucle principal de generación

Iteramos sobre el dataset humano.
1. Leemos el artículo original.
2. Lo enviamos a Gemini.
3. Guardamos el resultado en una lista.
4. Aplicamos un `time.sleep` para respetar los límites de velocidad de la versión gratuita de la API.

Nota: Algunos de los artículos (aprox. el 0.1%) no pueden ser reescritos por el LLM debido a la violación de políticas de contenido al tratarse de artículos de índole sexual o violenta. Sin embargo, no supone ningún problema, se filtrarán en el notebook 3.

In [5]:
## 2.4. Bucle Principal (Con Diagnóstico)
import os

print("🚀 Cargando datos humanos...")
if not os.path.exists(INPUT_FILE):
    print(f"❌ Error CRÍTICO: No encuentro el archivo {INPUT_FILE}")
else:
    df_human = pd.read_csv(INPUT_FILE)
    total_filas = len(df_human)
    print(f"📊 Total de artículos a procesar: {total_filas}")

    # -----------------------------------------------------------
    # LÓGICA DE REANUDACIÓN
    # -----------------------------------------------------------
    processed_ids = set()
    if os.path.exists(OUTPUT_FILE):
        try:
            existing = pd.read_csv(OUTPUT_FILE)
            # Solo si el archivo tiene datos y la columna correcta
            if 'original_id' in existing.columns:
                processed_ids = set(existing['original_id'].unique())
                print(f"🔄 ARCHIVO DETECTADO: Ya existen {len(processed_ids)} artículos procesados.")
            else:
                print("⚠️ El archivo de salida existe pero no tiene la estructura correcta.")
        except Exception as e:
            print(f"⚠️ Error leyendo archivo existente (se ignorará): {e}")

    # COMPROBACIÓN DE SEGURIDAD
    if len(processed_ids) >= total_filas:
        print("\n🛑 ¡ATENCIÓN! El script se ha detenido porque PARECE QUE YA ACABASTE.")
        print(f"   -> Filas totales: {total_filas}")
        print(f"   -> Filas ya procesadas: {len(processed_ids)}")
        print("   ✅ SOLUCIÓN: Si quieres empezar de cero, BORRA el archivo:")
        print(f"   rm {OUTPUT_FILE}")
    
    else:
        print(f"▶️ Iniciando trabajo... Faltan {total_filas - len(processed_ids)} artículos.")

        # Header mode: Solo escribimos cabecera si el archivo NO existe
        header_mode = not os.path.exists(OUTPUT_FILE)

        for index, row in df_human.iterrows():
            # 1. SALTAR YA PROCESADOS
            if index in processed_ids:
                continue

            original_text = row['article']
            
            try:
                # 2. LLAMADA A LA API
                ai_text = rewrite_article(original_text)
                
                if ai_text:
                    # 3. GUARDADO INCREMENTAL
                    single_row = pd.DataFrame([{
                        'original_id': index,
                        'article': ai_text,
                        'label': 1
                    }])
                    
                    single_row.to_csv(OUTPUT_FILE, mode='a', header=header_mode, index=False)
                    header_mode = False 
                    
            except Exception as e:
                print(f"⚠️ Error en artículo {index}: {e}")

            # 4. PAUSA (IMPORTANTE)
            time.sleep(4)
            
            # Feedback visual
            if (index + 1) % 10 == 0:
                print(f"   ... Procesados {index + 1}/{total_filas}")

        print(f"\n✅ ¡Proceso finalizado! Datos en: {OUTPUT_FILE}")

🚀 Cargando datos humanos...
📊 Total de artículos a procesar: 5000
🔄 ARCHIVO DETECTADO: Ya existen 4344 artículos procesados.
▶️ Iniciando trabajo... Faltan 656 artículos.
⚠️ Error en artículo 2021: Invalid operation: The `response.parts` quick accessor requires a single candidate, but but `response.candidates` is empty.
This appears to be caused by a blocked prompt, see `response.prompt_feedback`: block_reason: PROHIBITED_CONTENT

   ... Procesados 4350/5000
   ... Procesados 4360/5000
   ... Procesados 4370/5000
   ... Procesados 4380/5000
   ... Procesados 4390/5000
   ... Procesados 4400/5000
   ... Procesados 4410/5000
   ... Procesados 4420/5000
   ... Procesados 4430/5000
   ... Procesados 4440/5000
   ... Procesados 4450/5000
   ... Procesados 4460/5000
   ... Procesados 4470/5000
   ... Procesados 4480/5000
   ... Procesados 4490/5000
   ... Procesados 4500/5000
   ... Procesados 4510/5000
   ... Procesados 4520/5000
   ... Procesados 4530/5000
   ... Procesados 4540/5000
   ..

In [6]:
# CHECK LIST OF MODELS AVAILABLE
models = genai.list_models()
for m in models:
    print(m.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
mod